In [ ]:
This is a notebook to document import python routines I have written

# Reading Files

I have written routines to read lammps data files, lammps trajectories, and xyz files. I am working on a routine to read H5MD trajectories. The routines are stored in the conf_tools.py file which can be accessed with a simple import command.

In [3]:
from conf_tools import *

The routine for reading lammps data files is shown below.

In [4]:
def readConf(file, cast=True):

  atoms = []
  box = []
  bonds = []
  header = 'header'
  for line in file:
    L = line.split()
    if len(L) > 0 and L[0] in ['Atoms','Velocities','Bonds']:
      header = L[0]
    if len(L) > 0 and L[-1] in ['xhi','yhi','zhi']:
      box.append(L[:2])
    elif len(L) > 4 and header == 'Atoms':
      atoms.append(L)
    elif len(L) > 3 and header == 'Bonds':
      bonds.append(L)

  if cast:
    box = np.array(box,dtype=float)
    atoms = np.array(atoms,dtype=float)
    bonds = np.array(bonds,dtype=int)
      
  return box, atoms, bonds

The routine for reading lammps trajectory files is shown below

In [ ]:
def readTrj(file, nCols=5):

  """ default """
  time = None
  nAtoms = 0
  box = None 
  atoms = None
  item = None

  line = file.readline().strip()
  while (line != ''):
    L = line.split()
    if L[0] == 'ITEM:':
      item = L[1]
    if item is not None:
      if item == 'TIMESTEP':
        line = file.readline().strip()
        time = int(line)
      elif item == 'NUMBER':
        line = file.readline().strip()
        nAtoms = int(line)
      elif item == 'BOX':
        box = np.fromfile(file, float, 6, ' ')
        box = box.reshape((3,2))
      elif item == 'ATOMS':
        atoms = np.fromfile(file, float, nAtoms*nCols, ' ')
        atoms = atoms.reshape((nAtoms, nCols))
        break
      else:
        continue
    line = file.readline().strip()

  return time, box, atoms